In [ ]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import os

# 设置关节顺序 (按照 right_hip_pitch_joint, right_knee_joint, right_ankle_joint 的顺序)
joint_order = ['right_hip_pitch_joint', 'right_knee_joint', 'right_ankle_joint']

# 列出可用的CSV文件
raw_csv_dir = Path("../../data/Atom3motors/raw_csv")
csv_files = list(raw_csv_dir.glob("*.csv"))
print("可用的CSV文件:")
for i, csv_file in enumerate(csv_files):
    print(f"{i}: {csv_file.name}")

# 选择要转换的文件 (修改这里选择不同的文件索引)
selected_index = 2  # 0: steps_10s, 1: sin5Hz_10s, 2: chrip_20s
csv_path = csv_files[selected_index]

print(f"\n选择的文件: {csv_path.name}")
df = pd.read_csv(csv_path)
print(f"数据形状: {df.shape}")
print(f"列名: {df.columns.tolist()}")

In [ ]:
# 根据关节顺序重新排列数据
print("\n根据关节顺序重新排列数据...")
print(f"关节顺序: {joint_order}")

# 提取time列
time_data = torch.tensor(df['time'].values, dtype=torch.float32)

# 为每个关节提取数据并按照指定顺序排列
num_joints = len(joint_order)
num_samples = len(df)

# 初始化tensor
des_dof_pos = torch.zeros((num_samples, num_joints), dtype=torch.float32)
dof_pos = torch.zeros((num_samples, num_joints), dtype=torch.float32)
dof_vel = torch.zeros((num_samples, num_joints), dtype=torch.float32)

# 按照关节顺序填充数据
for i, joint_name in enumerate(joint_order):
    des_col = f'des_dof_pos_{joint_name}'
    pos_col = f'dof_pos_{joint_name}'
    vel_col = f'dof_vel_{joint_name}'
    
    if des_col in df.columns:
        des_dof_pos[:, i] = torch.tensor(df[des_col].values, dtype=torch.float32)
    if pos_col in df.columns:
        dof_pos[:, i] = torch.tensor(df[pos_col].values, dtype=torch.float32)
    if vel_col in df.columns:
        dof_vel[:, i] = torch.tensor(df[vel_col].values, dtype=torch.float32)

print(f"time shape: {time_data.shape}")
print(f"des_dof_pos shape: {des_dof_pos.shape}")
print(f"dof_pos shape: {dof_pos.shape}")
print(f"dof_vel shape: {dof_vel.shape}")
print(f"关节数量: {num_joints}")
print(f"时间步数: {num_samples}")

In [ ]:
# 保存为PT文件
output_dir = Path("../../data/Atom3motors")
output_dir.mkdir(parents=True, exist_ok=True)

# 根据输入文件名生成输出文件名
input_filename = csv_path.stem  # 获取不带扩展名的文件名
output_filename = f"{input_filename}.pt"
output_path = output_dir / output_filename

torch.save({
    "time": time_data,
    "dof_pos": dof_pos,
    "des_dof_pos": des_dof_pos,
    "dof_vel": dof_vel,
}, output_path)

print(f"数据已保存到: {output_path}")
print(f"文件大小: {output_path.stat().st_size / 1024:.2f} KB")
print(f"数据格式: time({time_data.shape}), dof_pos({dof_pos.shape}), des_dof_pos({des_dof_pos.shape}), dof_vel({dof_vel.shape})")

In [ ]:
# 绘制轨迹对比图
plt.figure(figsize=(16, 12))

# 为每个关节创建子图
for i, joint_name in enumerate(joint_order):
    plt.subplot(num_joints, 2, 2*i + 1)
    
    # 位置对比
    plt.plot(time_data.numpy(), des_dof_pos[:, i].numpy(), 'b--', label=f'Target {joint_name}', linewidth=1.5, alpha=0.8)
    plt.plot(time_data.numpy(), dof_pos[:, i].numpy(), 'r-', label=f'Actual {joint_name}', linewidth=1)
    plt.xlabel('Time [s]')
    plt.ylabel('Position [rad]')
    plt.title(f'{joint_name} - Position')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(num_joints, 2, 2*i + 2)
    
    # 速度
    plt.plot(time_data.numpy(), dof_vel[:, i].numpy(), 'g-', label=f'Velocity {joint_name}', linewidth=1)
    plt.xlabel('Time [s]')
    plt.ylabel('Velocity [rad/s]')
    plt.title(f'{joint_name} - Velocity')
    plt.legend()
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plot_path = output_dir / f"{input_filename}_trajectory.png"
plt.savefig(plot_path, dpi=150, bbox_inches='tight')
print(f"图表已保存到: {plot_path}")
plt.show()